In [ ]:
import glob
from PIL import Image
from torch.utils.data import Dataset
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
import torchvision

In [ ]:
from torch.utils.data import Dataset
import numpy as np

class Dataset(Dataset):
    def __init__(self, root_dir, partition, size=40, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.partition = partition

        self.img1_path = os.listdir('%s/%s/1/' % (self.root_dir, partition))
        self.mask_path = os.listdir('%s/%s/mask/' % (self.root_dir, partition))

    def __len__(self):
        return len(self.img1_path)

    def __getitem__(self, idx):
        
        img1 = Image.open('%s/%s/1/%s' % (self.root_dir, self.partition, self.img1_path[idx]))
        mask = np.array(Image.open('%s/%s/mask/%s' % (self.root_dir, self.partition, self.mask_path[idx])))
        
        if self.transform:
            img1 = self.transform(img1)
            mask = self.transform(mask)

        return img1, mask

In [ ]:
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.transforms import Compose, ToTensor, RandomAutocontrast, RandomAdjustSharpness, Normalize, RandomHorizontalFlip, RandomAutocontrast, RandomVerticalFlip

transform = transforms.Compose([ToTensor(),
                              transforms.Normalize([0.2823, 0.2935, 0.2716],[0.1515, 0.1607, 0.1534])])

transform_1 = transforms.Compose([RandomHorizontalFlip(p=0.5), RandomVerticalFlip(p=0.5)])

In [ ]:
desination_dir = '/content/2d/'
batch_size = 32

dataset = Dataset(desination_dir, partition='train', transform = transform_1)

trainset, valset = torch.utils.data.random_split(dataset, [int(len(dataset)*0.8), int(len(dataset)) - int(len(dataset)*0.8)])

train_loader = DataLoader(trainset, batch_size = batch_size, shuffle = False)
val_loader = DataLoader(valset, batch_size = batch_size, shuffle = False)

In [ ]:
dataset[0][0].shape

In [ ]:
# структура сети 

from torch import nn

class Unet(nn.Module):
    def __init__(self, channels = 16):
        super(Unet, self).__init__()
        self.channels = channels
        
        self.block1 = nn.Sequential(
            nn.Conv2d(3, self.channels, kernel_size=3, padding = 1), # n=224
            nn.ReLU(), # n=222
            nn.Conv2d(self.channels, self.channels, kernel_size=3, padding = 1), # n=224
            nn.ReLU()) # n=220

        self.block2 = nn.Sequential(
            nn.MaxPool2d((2, 2)), #n=110
            nn.Conv2d(self.channels, self.channels * 2, kernel_size=3, padding = 1), # n=112
            nn.ReLU(), # n=3
            nn.Conv2d(self.channels * 2, self.channels * 2, kernel_size=3, padding = 1), # n=112
            nn.ReLU())

        self.block3 = nn.Sequential(
            nn.MaxPool2d((2, 2)), #n=53
            nn.Conv2d(self.channels * 2, self.channels * 2 * 2, kernel_size=3, padding = 1), # n=56
            nn.ReLU(), # n=3
            nn.Conv2d(self.channels * 4, self.channels * 4, kernel_size=3, padding = 1), # n=56
            nn.ReLU(),
            nn.ConvTranspose2d(self.channels * 4, self.channels * 4, kernel_size = 3, stride = 2, padding = 1, output_padding=1)) #n=50
        
        self.block4 = nn.Sequential(
            nn.Conv2d(self.channels * 6, self.channels * 4, kernel_size=3, padding = 1), 
            nn.ReLU(),
            nn.Conv2d(self.channels * 4, self.channels * 2, kernel_size=3, padding = 1),
            nn.ReLU(),
            nn.ConvTranspose2d(self.channels * 2, self.channels * 2, kernel_size = 3, stride = 2, padding = 1, output_padding=1)
        )
        self.block5 = nn.Sequential(
            nn.Conv2d(self.channels * 3, self.channels * 2, kernel_size=3, padding = 1), 
            nn.ReLU(),
            nn.Conv2d(self.channels * 2, self.channels, kernel_size=3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(self.channels, 1, kernel_size=1),
            nn.ReLU()
        )


    def forward(self, x):
        #print("x", x.shape) #[4, 3, 224, 224]
        x1 = self.block1(x) #[4, 64, 224, 224]
        #print("x1", x1.shape)
        x2 = self.block2(x1) #[4, 128, 112, 112]
        #print("x2", x2.shape)
        x3 = self.block3(x2) #[4, 256, 112, 112]
        #print("x3", x3.shape)
        x4 = torch.cat((x2, x3), dim = 1) #[4, 384, 112, 112]
        #print("x4",x4.shape)
        x5 = self.block4(x4) #[4, 128, 224, 224]
        #print("x5", x5.shape)
        x6 = torch.cat((x5, x1), dim = 1) #
        #print("x6", x6.shape) #[4, 192, 224, 224]
        x7 = self.block5(x6)
        #print("x7", x7.shape)
        return x7 # predicted map

In [ ]:
#Loss
class Loss(nn.Module):

    def __init__(self, epsilon=1e-6):
        super().__init__()
        self.epsilon = epsilon

    def forward(self, predict, target):
        predict = predict.flatten(1)
        target = target.flatten(1)

        loss = torch.sum(torch.abs(torch.sub(predict, target))) + self.epsilon
        #print("loss", loss)
      
        return loss.mean()  # over batch

In [ ]:
criterion = Loss()

In [ ]:
def train(model, criterion, optimizer, epoch=None, masks_in_progress=[]):
    ep_loss = 0
    model.train()
    for img_batch, masks_batch in train_loader:
        optimizer.zero_grad()
        output = model(img_batch.to(device))
        loss = criterion(output, masks_batch.to(device))
        loss.backward()
        optimizer.step()
        ep_loss += loss.item()

    val_loss = 0
    for i, batch in enumerate(val_loader):
        with torch.no_grad():
            img_batch, masks_batch = batch
            output = model(img_batch.to(device))
            loss = criterion(output, masks_batch.to(device))
            val_loss += loss.item()
            if i == 0:
                masks_in_progress.append(output[0].cpu())

    print("Epoch {} Train loss {:.2f} Val loss {:.2f}".format(epoch, ep_loss/len(train_loader), val_loss/len(val_loader)))

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
model = Unet().to(device)
criterion = Loss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-4)
with torch.no_grad():
    masks_in_progress = [model(next(iter(val_loader))[0].to(device))[0].cpu()]

In [ ]:
for epoch in range(15):
    train(model, criterion, optimizer, epoch, masks_in_progress)

In [ ]:
torch.cuda.empty_cache()